In [1]:
import keras.preprocessing.text as kpt
import pandas as pd


/Users/abhsharm/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
raw_train = pd.read_csv("train.csv")
raw_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

y = raw_train[labels]
y=y.values
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [37]:
X = []
X = raw_train['comment_text']


In [35]:
# https://stackoverflow.com/questions/36700790/keras-text-classification-lstm-how-to-input-text

X[0] = raw_train['comment_text']

#makes list of words in bag-of-all-words-in-dataset for numerical of 
tk = keras.preprocessing.text.Tokenizer(nb_words=2000, lower=True, split=" ")
tk.fit_on_texts(X[0]) #finds all words

/Users/nicksteinhauser/anaconda/envs/py36/lib/python3.6/site-packages/keras/preprocessing/text.py:157: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [46]:
x = tk.texts_to_sequences(X)


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ..., 
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [40]:
from pyspark.sql import SparkSession

In [54]:
spark = SparkSession.builder.appName('caustic').getOrCreate()

In [130]:
# schema necessary for label columns to be ints for vectorization later
start_data = spark.read.csv("train.csv", header = "true", schema = "id STRING, comment_text STRING, toxic INT, severe_toxic INT, obscene INT,threat INT,insult INT,identity_hate INT")
start_data = start_data.na.drop() # label columns have missing values

In [131]:
from pyspark.sql.functions import length
data = start_data.withColumn('length', length(start_data['comment_text']))

In [132]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

# Create all the features to the data set
tokenizer = Tokenizer(inputCol="comment_text", outputCol="token_text") #can cut down on columns
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')


In [133]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up_features = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')
clean_up_labels = VectorAssembler(inputCols=labels, outputCol='labels')

In [134]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[ tokenizer, stopremove, hashingTF, idf, clean_up_features, clean_up_labels])

In [135]:
cleaner = data_prep_pipeline.fit(data)
cleaned = cleaner.transform(data)

In [138]:
# Show label of and resulting features

cleaned.select(['labels', 'features']).show()

+--------------------+--------------------+
|              labels|            features|
+--------------------+--------------------+
|           (6,[],[])|(262145,[19208,24...|
|           (6,[],[])|(262145,[17429,38...|
|           (6,[],[])|(262145,[14,9639,...|
|           (6,[],[])|(262145,[37852,39...|
|           (6,[],[])|(262145,[44501,72...|
|[1.0,1.0,1.0,0.0,...|(262145,[25551,34...|
|           (6,[],[])|(262145,[37470,55...|
|           (6,[],[])|(262145,[5147,523...|
|           (6,[],[])|(262145,[9639,916...|
|           (6,[],[])|(262145,[13781,21...|
|       (6,[0],[1.0])|(262145,[9639,122...|
|           (6,[],[])|(262145,[70503,87...|
|           (6,[],[])|(262145,[29238,45...|
|           (6,[],[])|(262145,[15889,19...|
|           (6,[],[])|(262145,[14,9639,...|
|           (6,[],[])|(262145,[56614,86...|
|           (6,[],[])|(262145,[4649,158...|
|           (6,[],[])|(262145,[9639,112...|
|           (6,[],[])|(262145,[24417,25...|
|           (6,[],[])|(262145,[5

[Row(id="Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms", comment_text=' just closure on some GAs after I voted at New York Dolls FAC. And please don\'t remove the template from the talk page since I\'m retired now.89.205.38.27"', toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0, length=154, token_text=['', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fac.', 'and', 'please', "don't", 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', "i'm", 'retired', 'now.89.205.38.27"'], stop_tokens=['', 'closure', 'gas', 'voted', 'new', 'york', 'dolls', 'fac.', 'please', 'remove', 'template', 'talk', 'page', 'since', 'retired', 'now.89.205.38.27"'], hash_token=SparseVector(262144, {19208: 1.0, 24417: 1.0, 25000: 1.0, 29945: 1.0, 46075: 1.0, 59853: 1.0, 72125: 1.0, 82999: 1.0, 83922: 1.0, 91677: 1.0, 97171: 1.0, 100258: 1.0, 101169: 1.0, 103838: 2.0, 110427: 1.

AttributeError: 'DataFrame' object has no attribute 'cast'